<a href="https://colab.research.google.com/github/ogane0112/Y_scraiping/blob/main/Data_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=f5351e48660211884ba72df4575a1a42da59acb9e94a9d488c9bf72bd9f25828
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
!pip install request

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


In [ ]:
from datetime import datetime
import openpyxl as px
from openpyxl.styles import PatternFill
import requests
from bs4 import BeautifulSoup

#プログラム2｜ヤフオクページから情報をスクレイピング
def Pagecrawling(soup, item_list):

    # プログラム2-1｜<li>タグのなかで、class='Product'のものを変数productsに格納
    products = soup.find_all('li', class_='Product')

    # プログラム2-2｜変数productsの要素を一つずつ調査
    for product in products:
        # プログラム2-3｜変数productsのなかで、aタグで、class='Product__titleLink'のものを変数urlsに格納
        urls = product.find_all('a', class_='Product__titleLink')

        # プログラム2-4｜変数productsのなかで、divタグで、class='Product__priceInfo'のものを変数pricesに格納
        prices = product.find_all('div', class_='Product__priceInfo')

        # プログラム2-5｜変数productsのなかで、divタグで、class='Product__otherInfo'のものを変数othersに格納
        others = product.find_all('div', class_='Product__otherInfo')

        # プログラム2-6｜zip関数でまとめて繰り返し処理を実行
        for (url, price, other) in zip(urls, prices, others):

            # プログラム2-7｜変数linkに変数urlのhref部分を取得（製品url）
            link = url.get('href')

            # プログラム2-8｜変数nameに変数urlのtitle部分を取得（製品名）
            name = url.get('title')

            # プログラム2-9｜変数priceのspanタグで、class='Product__price'のものを変数pricevaluesに格納
            pricevalues = price.find_all('span', class_='Product__price')

            # プログラム2-10｜変数の初期化
            currentprice = '-'
            fixedprice = '-'

            # プログラム2-11｜変数pricevaluesの要素を一つずつ処理
            for pricevalue in pricevalues:

                # プログラム2-12｜もし変数pricevalueのテキスト情報が'現在'が含まれていれば
                if '現在' in pricevalue.get_text():

                    # プログラム2-13｜変数currentpriceに現在の価格を取得（'\n'で改行を削除）
                    currentprice = pricevalue.get_text().replace('\n', '')

                # プログラム2-14｜もし変数pricevalueのテキスト情報が'即決'が含まれていれば
                elif '即決' in pricevalue.get_text():

                    # プログラム2-15｜変数fixedpriceに即決の価格を取得（'\n'で改行を削除）
                    fixedprice = pricevalue.get_text().replace('\n', '')

            # プログラム2-16｜変数labelのdivタグの0番目の要素を取得（'\n'で改行を削除）→入札
            label = other.find_all('div')[0].get_text().replace('\n','')

            # プログラム2-17｜変数labelのdivタグの1番目の要素を取得（'\n'で改行を削除）→残り時間
            lefttime = other.find_all('div')[1].get_text().replace('\n','')

            # プログラム2-18｜リスト「item_list」に必要な要素を追加
            item_list.append([name, link, currentprice, fixedprice, label, lefttime])

    # プログラム2-19｜リスト「item_list」を返す
    return item_list

# プログラム3｜次へリンクを探す
def Checkpage(pagerlinks):

    # プログラム3-1｜変数pageurlをNoneにリセット
    pageurl = None

    # プログラム3-2｜引数pagerlinksの中に、「次へ」があれば、そのurlをpageurlに設定（「次へ」がない場合は、Noneのまま）
    for pagerlink in pagerlinks:
        if pagerlink.get_text() == '次へ':
            pageurl = pagerlink.get('href')
            break

    # プログラム3-3｜pageurlを返す
    return  pageurl

# プログラム4｜エクセルに出力
def Write_excel(item_list, keyword):

    # プログラム4-1｜エクセルを取得
    wb = px.Workbook()
    ws = wb.active

    # プログラム4-2｜エクセルのヘッダーの背景色を設定
    fill = PatternFill(patternType='solid', fgColor='e0e0e0', bgColor='e0e0e0')

    # プログラム4-3｜エクセル1行目のヘッダーを出力
    headers = ['No', 'タイトル', 'URL', '現在の価格','即決価格','入札','残り時間']
    for i, header in enumerate(headers):
        ws.cell(row=1, column=1+i, value=headers[i])
        ws.cell(row=1, column=1+i).fill = fill

    # プログラム4-4｜エクセル2行目以降のデータを出力
    for y, row in enumerate(item_list):
        ws.cell(row= y+2, column= 1, value= y+1)
        for x, cell in enumerate(row):
            if x == 0:
                ws.cell(row= y+2, column= x+2, value=item_list[y][x])
            elif x == 1:
                ws.cell(row= y+2, column= x+1).hyperlink = item_list[y][x]
                ws.cell(row= y+2, column= x+1).font = px.styles.fonts.Font(color='0000EE')
            else:
                ws.cell(row= y+2, column= x+1, value=item_list[y][x])

    # プログラム4-5｜日付を取得
    now = datetime.now()
    hiduke = now.strftime('%Y-%m-%d')

    # プログラム4-6｜エクセルファイルの保存
    filename = hiduke + '_' + keyword  + '_' +'YahooAuction.xlsx'
    wb.save(filename)

#プログラム5｜mainプログラム
# プログラム5-1｜検索キーワードとYahooオークションURLの設定
keyword = 'Python'
url = 'https://auctions.yahoo.co.jp/search/search?p=' + keyword + '&n=100';

# プログラム5-2｜ヤフオクページの取得
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

# プログラム5-3｜取得したデータを格納するリスト
item_list = []

# プログラム5-4｜リスト「item_list」にプログラム3の結果を格納
item_list = Pagecrawling(soup, item_list)

# プログラム5-5｜プログラム4を実行して、次のページがあるかどうか調査
pageurl = Checkpage(soup.find_all('a', class_='Pager__link'))

# プログラム5-6｜次のページがなくなるまで、処理を実行
while True:
    # プログラム5-7｜次のページがないならwhile文を終了
    if pageurl is None:
        break

    # プログラム5-8｜次のページがあるなら処理を実行
    else:
        # プログラム5-9｜次のページを取得
        nextpage = requests.get(pageurl)
        soup = BeautifulSoup(nextpage.text, 'lxml')

        # プログラム5-10｜リスト「item_list」にプログラム3の結果を格納
        item_list = Pagecrawling(soup, item_list)

        # プログラム5-11｜プログラム4を実行して、次のページがあるかどうか調査
        pageurl = Checkpage(soup.find_all('a', class_='Pager__link'))

# プログラム5-12｜リスト「item_list」の中身を調べる
[print(i, item) for i, item in enumerate(item_list)]

# プログラム5-13｜プログラム5を実行して、リスト「item_list」をエクセルに書き出す
Write_excel(item_list, keyword)

0 ['MGC モデルガン コルトパイソン 2.5インチ SRHW 木製グリップ スーパーリアルヘヴィーウェイト COLT PYTHON SPG 現状品【20', 'https://page.auctions.yahoo.co.jp/jp/auction/f1095229954', '現在3,630円', '-', '入札10', '残り3日']
1 ['東京マルイ ガスリボルバー コルト パイソン 4インチ COLT PYTHON 現状品【20', 'https://page.auctions.yahoo.co.jp/jp/auction/u1095479957', '現在1,100円', '-', '入札0', '残り5日']
2 ['●美品●極上●PYTHON●パイソン●ブランド●ブラウン●ハンドバッグ●バッグ●メンズ●レディース●お洒落●GA0065', 'https://page.auctions.yahoo.co.jp/jp/auction/j1094521494', '現在787円', '-', '入札2', '残り3日']
3 ['TOKYO MARUI 東京マルイ PYTHON 357 ジャンク品', 'https://page.auctions.yahoo.co.jp/jp/auction/1095105998', '現在1,400円', '-', '入札5', '残り6時間']
4 ['MGC COLT PYTHON 357 MAGNUM コルト パイソン 357 マグナム RODY 44 ＭAGUNUM ホルスター付き モデルガン エムジーシー SPG刻印あり', 'https://page.auctions.yahoo.co.jp/jp/auction/s1095466490', '現在1,000円', '-', '入札1', '残り4日']
5 ['COLT PYTHON・357MAGNUM\u3000357マグナム\u3000ジャンク', 'https://page.auctions.yahoo.co.jp/jp/auction/m1095262860', '現在2,100円', '-', '入札7', '残り1日']
6 ['230526PT010089 PYTHON 357 MAGNUM CTG モデルガン\u3000ジャンク品',

NameError: ignored

In [ ]:
from datetime import datetime
import openpyxl as px
from openpyxl.styles import PatternFill
import requests
from bs4 import BeautifulSoup
import gradio as gr


# Function to perform page crawling
def Pagecrawling(soup, item_list):
    products = soup.find_all('li', class_='Product')

    for product in products:
        item = {}
        item['title'] = product.find('a', class_='Product__titleLink').get_text()
        item['link'] = product.find('a', class_='Product__titleLink').get('href')
        item['price'] = product.find('span', class_='Product__priceValue').get_text()
        item_list.append(item)

    return item_list


# Function to check for next page link
def Checkpage(pagerlinks):
    pageurl = None

    for pagerlink in pagerlinks:
        if pagerlink.get_text() == '次へ':
            pageurl = pagerlink.get('href')
            break

    return pageurl


# Function to write data to Excel
def Write_excel(item_list, keyword):
    wb = px.Workbook()
    ws = wb.active
    ws.title = "Yahoo Auction Data"

    # Write header
    header = ['Title', 'Link', 'Price']
    ws.append(header)

    # Write data
    for item in item_list:
        row = [item['title'], item['link'], item['price']]
        ws.append(row)

    # Save the workbook
    filename = f"yahoo_auction_{keyword}_{datetime.now().strftime('%Y%m%d%H%M%S')}.xlsx"
    wb.save(filename)

    return filename


# Function to run the scraping and Excel writing process
def run_scraping(keyword):
    url = 'https://auctions.yahoo.co.jp/search/search?p=' + keyword + '&n=100'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    item_list = []
    item_list = Pagecrawling(soup, item_list)
    pageurl = Checkpage(soup.find_all('a', class_='Pager__link'))

    while True:
        if pageurl is None:
            break
        else:
            nextpage = requests.get(pageurl)
            soup = BeautifulSoup(nextpage.text, 'lxml')
            item_list = Pagecrawling(soup, item_list)
            pageurl = Checkpage(soup.find_all('a', class_='Pager__link'))

    filename = Write_excel(item_list, keyword)
    return f"Scraping completed. Data saved to {filename}."


# Gradio interface
def scraping_interface(keyword):
    result = run_scraping(keyword)
    return result


# Create a Gradio interface
iface = gr.Interface(fn=scraping_interface, inputs="text", outputs="text", title="Yahoo Auction Scraper")
iface.launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 k

In [ ]:
from datetime import datetime
import openpyxl as px
from openpyxl.styles import PatternFill
import requests
from bs4 import BeautifulSoup
import gradio as gr

# プログラム2-19｜スクレイピング関数
def scrape(keyword):
    # プログラム5-1｜検索キーワードとYahooオークションURLの設定
    url = 'https://auctions.yahoo.co.jp/search/search?p=' + keyword + '&n=100'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    # プログラム5-4｜リスト「item_list」にプログラム3の結果を格納
    item_list = Pagecrawling(soup, [])

    # プログラム5-5｜プログラム4を実行してエクセルに出力
    Write_excel(item_list, keyword)

    return item_list

# プログラム5-6｜mainプログラム
def main(keyword):
    # プログラム5-13｜スクレイピングを実行
    item_list = scrape(keyword)

    # プログラム5-12｜リスト「item_list」の中身を調べる
    [print(i, item) for i, item in enumerate(item_list)]

# プログラム2｜ヤフオクページから情報をスクレイピング
def Pagecrawling(soup, item_list):
    # ...

# プログラム4｜エクセルに出力
def Write_excel(item_list, keyword):
    # ...

# グラディオのインターフェース
iface = gr.Interface(fn=main, inputs="text", outputs=None, title="Yahoo Auction Scraper", description="Specify the keyword to scrape Yahoo Auctions.")
iface.launch()


IndentationError: ignored